In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
#from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

In [2]:
df1=pd.read_csv("engaged.csv")
df2=pd.read_csv("notEngaged.csv")

In [3]:
dft=pd.concat([df1,df2])

In [4]:
y_train=dft["Result"]
x_train=dft.drop("Result",axis=1)

In [5]:
model = LogisticRegression()
model.fit(x_train, y_train)


/home/prashanth/Projects/Environments/project1_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [6]:
joblib.dump(model, 'model_joblib_logistic_regression')

['model_joblib_logistic_regression']

In [7]:
mj = joblib.load('model_joblib_logistic_regression')

In [8]:
y_pred = mj.predict(x_train)

In [9]:
count=0

In [10]:
y_train=y_train.to_numpy()

In [11]:
for i in range(len(y_pred)):
    if(y_pred[i]==y_train[i]):
        count=count+1

In [12]:
print(count/len(y_pred))

0.9689929571023507
